In [1]:
from datetime import datetime
import pandas as pd
import ast
import numpy as np
import itertools
from pandas.api.types import CategoricalDtype

In [36]:
file_users = "/Users/kadir/github-data/Users.csv"
file_watcher = "/Users/kadir/github-data/Watchers.csv"
file_repos = "/Users/kadir/github-data/Repos.csv"
file_commits = "/Users/kadir/github-data/Commits.csv"
file_commit_comments = "/Users/kadir/github-data/Commit_comments.csv"
file_issues = "/Users/kadir/github-data/Issues.csv"
file_issue_comments = "/Users/kadir/github-data/Issue_comments.csv"
file_pull_requests = "/Users/kadir/github-data/Pull_requests.csv"
file_pull_request_comments = "/Users/kadir/github-data/Pull_request_comments.csv"
df_users = pd.read_csv(file_users, low_memory=False, na_filter=False,na_values=0)

df_repos = pd.read_csv(file_repos, low_memory=False, na_filter=False,na_values=0)
df_repos['language'] = df_repos['language'].astype(str)
df_repos['language'] = df_repos['language'].str.lower()
df_repos['language'].replace('', np.nan, inplace=True)

df_watched_repos = pd.read_csv(file_watcher, low_memory=False, na_filter=False, na_values=0)

df_commits = pd.read_csv(file_commits, low_memory=False, na_filter=False, na_values=0)
df_commit_comments = pd.read_csv(file_commit_comments, low_memory=False, na_filter=False, na_values=0)

df_issues = pd.read_csv(file_issues, low_memory=False, na_filter=False, na_values=0)
df_issue_comments = pd.read_csv(file_issue_comments, low_memory=False, na_filter=False, na_values=0)
df_issues['closed_by_id'] = df_issues['closed_by_id'].astype(str)
df_issues['closed_by_id'].replace("",0,inplace=True)
df_issues['closed_by_id'] = df_issues['closed_by_id'].astype(int)

df_issues['assignee_id'] = df_issues['assignee_id'].astype(str)
df_issues['assignee_id'].replace("",0,inplace=True)
df_issues['assignee_id'] = df_issues['assignee_id'].astype(int)
df_issues.hasPullRequest = df_issues.hasPullRequest.astype(int)

df_pull_requests = pd.read_csv(file_pull_requests, low_memory=False, na_filter=False, na_values=0)
df_pull_requests['merged_by_id'] = df_pull_requests['merged_by_id'].astype(str)
df_pull_requests['merged_by_id'].replace("",0,inplace=True)
df_pull_requests['merged_by_id'] = df_pull_requests['merged_by_id'].astype(int)

df_pull_requests['assignee_id'] = df_pull_requests['assignee_id'].astype(str)
df_pull_requests['assignee_id'].replace("",0,inplace=True)
df_pull_requests['assignee_id'] = df_pull_requests['assignee_id'].astype(int)

df_pull_request_comments = pd.read_csv(file_pull_request_comments, low_memory=False, na_filter=False, na_values=0)


In [37]:
def calculate_feature(dataframe,user_related_field,feature_name):
    mydict = dataframe[
        dataframe[user_related_field].isin(list(df_users.id))].groupby('repo_id')[user_related_field].value_counts().to_dict() # 100 user  
    df = pd.Series(mydict).reset_index()
    df.columns = ['repo_id', 'user_id', feature_name]  
    return df

In [38]:
def calculate_feature_sum(dataframe,user_related_field,feature_name):
    mydict = dataframe[
        dataframe['user_id'].isin(list(df_users.id))].groupby(['repo_id','user_id'])[user_related_field].sum().to_dict()
    df = pd.Series(mydict).reset_index()
    df.columns = ['repo_id', 'user_id', feature_name]
    return df

In [39]:
df_num_commit_committed = calculate_feature(df_commits,'committer_id','num_commit_committed')
df_num_commit_authored  = calculate_feature(df_commits,'author_id','num_commit_authored')
df_num_commit_commented = calculate_feature(df_commit_comments,'user_id','num_commit_commented')

df_num_issue_opened       = calculate_feature(df_issues,'user_id','num_issue_opened')
df_num_issue_closed       = calculate_feature(df_issues,'closed_by_id','num_issue_closed')
df_num_issue_assigned     = calculate_feature(df_issues,'assignee_id','num_issue_assigned')
df_num_issue_commented    = calculate_feature(df_issue_comments,'user_id','num_issue_commented')
df_num_issue_closedwithPR = calculate_feature_sum(df_issues,'hasPullRequest','num_issue_closedwithPR')

df_num_pr_opened   = calculate_feature(df_pull_requests,'user_id','num_pr_opened')
df_num_pr_merged   = calculate_feature(df_pull_requests,'merged_by_id','num_pr_merged')
df_num_pr_assigned = calculate_feature(df_pull_requests,'assignee_id','num_pr_assigned')
df_num_pr_comments = calculate_feature(df_pull_request_comments,'user_id','num_pr_commented')


In [40]:
combinations = list(itertools.product(df_repos.repo_id.sort_values(),df_users.id.sort_values()))
df_all_user_repo = pd.DataFrame(data=combinations, columns = ['repo_id', 'user_id'])
df_all_user_repo.head()

,repo_id,user_id
0,1,21
1,1,25
2,1,38
3,1,108
4,1,173


In [41]:
ownership = df_repos[['repo_id','owner_id']]
x = pd.merge(df_all_user_repo, ownership, how="left", on=['repo_id'])
x['is_ownered'] = np.where((x['user_id'] == x['owner_id']),1,0)
x.head()

,repo_id,user_id,owner_id,is_ownered
0,1,21,1,0
1,1,25,1,0
2,1,38,1,0
3,1,108,1,0
4,1,173,1,0


In [42]:
x = pd.merge(x[['repo_id','user_id','is_ownered']], df_num_commit_committed, how="left", on=['repo_id', 'user_id'])
x = pd.merge(x, df_num_commit_authored, how="left", on=['repo_id', 'user_id'])
x = pd.merge(x, df_num_commit_commented,  how="left", on=['repo_id', 'user_id'])
x = pd.merge(x, df_num_issue_opened, how="left", on=['repo_id', 'user_id'])
x = pd.merge(x, df_num_issue_closed, how="left", on=['repo_id', 'user_id'])
x = pd.merge(x, df_num_issue_assigned, how="left", on=['repo_id', 'user_id'])
x = pd.merge(x, df_num_issue_commented, how="left", on=['repo_id', 'user_id'])
x = pd.merge(x, df_num_issue_closedwithPR, how="left", on=['repo_id', 'user_id'])
x = pd.merge(x, df_num_pr_opened, how="left", on=['repo_id', 'user_id'])
x = pd.merge(x, df_num_pr_merged, how="left", on=['repo_id', 'user_id'])
x = pd.merge(x, df_num_pr_assigned, how="left", on=['repo_id', 'user_id'])
features = pd.merge(x, df_num_pr_comments, how="left", on=['repo_id', 'user_id'])
features.head()

,repo_id,user_id,is_ownered,num_commit_committed,num_commit_authored,num_commit_commented,num_issue_opened,num_issue_closed,num_issue_assigned,num_issue_commented,num_issue_closedwithPR,num_pr_opened,num_pr_merged,num_pr_assigned,num_pr_commented
0,1,21,0,NaN,NaN,NaN,2.0,NaN,NaN,17.0,1.0,1.0,NaN,NaN,NaN
1,1,25,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,38,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,108,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,173,0,NaN,NaN,NaN,2.0,NaN,NaN,136.0,2.0,2.0,1.0,NaN,1.0


In [43]:
# Burayı çalıştırma. oy verilmeyen 0 olarak kabul edilir. NaN kalmalı !
#features.fillna(0, inplace=True) 
#features = features.astype(np.int64)
#features.head()

In [44]:
col_list= list(features)
col_list.remove('repo_id')
col_list.remove('user_id')
col_list
features['count'] = features[col_list].sum(axis=1)
features.head()

,repo_id,user_id,is_ownered,num_commit_committed,num_commit_authored,num_commit_commented,num_issue_opened,num_issue_closed,num_issue_assigned,num_issue_commented,num_issue_closedwithPR,num_pr_opened,num_pr_merged,num_pr_assigned,num_pr_commented,count
0,1,21,0,NaN,NaN,NaN,2.0,NaN,NaN,17.0,1.0,1.0,NaN,NaN,NaN,21.0
1,1,25,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
2,1,38,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
3,1,108,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
4,1,173,0,NaN,NaN,NaN,2.0,NaN,NaN,136.0,2.0,2.0,1.0,NaN,1.0,144.0


In [45]:
def calculate_total_features(dataframe,feature_name):
    series = dataframe['repo_id'].value_counts()
    df = pd.Series(series).reset_index()
    df.columns = ['repo_id',feature_name] 
    df = df.sort_values('repo_id')
    return df

In [46]:
df_commits_repo = calculate_total_features(df_commits,'total_commit')
df_commit_comments_repo = calculate_total_features(df_commit_comments,'total_commit_comment')

df_issues_repo = calculate_total_features(df_issues,'total_issue')
df_issue_comments_repo = calculate_total_features(df_issue_comments,'total_issue_comment')

df_pr_repo = calculate_total_features(df_pull_requests,'total_pr')
df_pr_comments_repo = calculate_total_features(df_pull_request_comments,'total_pr_comment')

In [47]:
features2 = pd.merge(features, df_commits_repo, how="left", on=['repo_id'])
features3 = pd.merge(features2, df_commit_comments_repo, how="left", on=['repo_id'])
features4 = pd.merge(features3, df_issues_repo, how="left", on=['repo_id'])
features5 = pd.merge(features4, df_issue_comments_repo, how="left", on=['repo_id'])
features6 = pd.merge(features5, df_pr_repo, how="left", on=['repo_id'])
features_total = pd.merge(features6, df_pr_comments_repo, how="left", on=['repo_id'])

In [48]:
features_total.head()

,repo_id,user_id,is_ownered,num_commit_committed,num_commit_authored,num_commit_commented,num_issue_opened,num_issue_closed,num_issue_assigned,num_issue_commented,...,num_pr_merged,num_pr_assigned,num_pr_commented,count,total_commit,total_commit_comment,total_issue,total_issue_comment,total_pr,total_pr_comment
0,1,21,0,NaN,NaN,NaN,2.0,NaN,NaN,17.0,...,NaN,NaN,NaN,21.0,NaN,2.0,187.0,491.0,102.0,20.0
1,1,25,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.0,NaN,2.0,187.0,491.0,102.0,20.0
2,1,38,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.0,NaN,2.0,187.0,491.0,102.0,20.0
3,1,108,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.0,NaN,2.0,187.0,491.0,102.0,20.0
4,1,173,0,NaN,NaN,NaN,2.0,NaN,NaN,136.0,...,1.0,NaN,1.0,144.0,NaN,2.0,187.0,491.0,102.0,20.0


In [49]:
#export_csv = features_total.to_csv (r'/Users/kadir/Desktop/featuresAll.csv',index=False, header=True)

In [50]:
#features_total.fillna(0, inplace=True)
#features_total.head()

In [51]:
features_total['O_num_commit_committed']= features_total['num_commit_committed']   / features_total['total_commit']
features_total['O_num_commit_authored'] = features_total['num_commit_authored']    / features_total['total_commit']
features_total['O_num_commit_commented']= features_total['num_commit_commented']   / features_total['total_commit_comment']

features_total['O_num_issue_opened']    = features_total['num_issue_opened']       / features_total['total_issue']
features_total['O_num_issue_closed']    = features_total['num_issue_closed']       / features_total['total_issue']
features_total['O_num_issue_assigned']  = features_total['num_issue_assigned']     / features_total['total_issue']
features_total['O_num_issue_closedPR']  = features_total['num_issue_closedwithPR'] / features_total['total_issue']
features_total['O_num_issue_commented'] = features_total['num_issue_commented']    / features_total['total_issue_comment']

features_total['O_num_pr_opened']       = features_total['num_pr_opened']        / features_total['total_pr']
features_total['O_num_pr_assigned']     = features_total['num_pr_assigned']       / features_total['total_pr']
features_total['O_num_pr_merged']       = features_total['num_pr_merged']        / features_total['total_pr']
features_total['O_num_pr_commented']    = features_total['num_pr_commented']     / features_total['total_pr_comment']

In [52]:
features_total.head()

,repo_id,user_id,is_ownered,num_commit_committed,num_commit_authored,num_commit_commented,num_issue_opened,num_issue_closed,num_issue_assigned,num_issue_commented,...,O_num_commit_commented,O_num_issue_opened,O_num_issue_closed,O_num_issue_assigned,O_num_issue_closedPR,O_num_issue_commented,O_num_pr_opened,O_num_pr_assigned,O_num_pr_merged,O_num_pr_commented
0,1,21,0,NaN,NaN,NaN,2.0,NaN,NaN,17.0,...,NaN,0.010695,NaN,NaN,0.053476,0.034623,0.009804,NaN,NaN,NaN
1,1,25,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,38,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,108,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,173,0,NaN,NaN,NaN,2.0,NaN,NaN,136.0,...,NaN,0.010695,NaN,NaN,0.106952,0.276986,0.019608,NaN,0.098039,0.05


In [53]:
def toplanguages(languages):
    languages = [x.lower() for x in languages]
    unique, counts = np.unique(languages, return_counts=True)
    languages_counts = dict(zip(unique, counts))
    x = {k: v for k, v in sorted(languages_counts.items(), key=lambda item: item[1],reverse=True)}
    toplanguages = list(x.keys())
    countlang = len(toplanguages)
    if countlang >= 10:
        bestLanguages = toplanguages[0:10]
    else:
        bestLanguages = toplanguages
    return bestLanguages

In [54]:
repos_lang_owned = df_repos[df_repos['owner_id'].isin(list(df_users.id))][['repo_id','owner_id','language']]
repos_lang_owned = repos_lang_owned.sort_values("owner_id")
repos_lang_owned.dropna(subset=['language'],inplace = True)
mydict = repos_lang_owned.groupby('owner_id').apply(lambda x: x['language'].tolist()).to_dict()
top_lang_owner = pd.Series(mydict).reset_index()
top_lang_owner.columns = ['user_id', 'languages']
top_lang_owner['top_lang_owned'] = top_lang_owner.languages.apply(toplanguages)
top_lang_owner.head()

,user_id,languages,top_lang_owned
0,21,"[c, ruby, ruby, ruby, go, ruby, ruby, go, ruby...","[ruby, javascript, go, coffeescript, c, c#, c+..."
1,25,"[javascript, ruby, javascript, javascript, rub...","[javascript, ruby, coffeescript, objective-c, ..."
2,38,"[ruby, javascript, javascript, coffeescript, c...","[ruby, javascript, coffeescript, shell, go, ob..."
3,108,"[ruby, shell, ruby, ruby, ruby, css, css, go, ...","[ruby, shell, go, javascript, css, coffeescrip..."
4,173,"[javascript, ruby, ruby, javascript, ruby, rub...","[ruby, javascript, shell, coffeescript, c, css..."


In [55]:
df_commitsx = pd.merge(df_commits,df_repos[['repo_id','language']],how="left", on=['repo_id'])
repos_lang_committed = df_commitsx[df_commitsx['committer_id'].isin(list(df_users.id))][['repo_id','committer_id','language']]
repos_lang_committed = repos_lang_committed.sort_values("committer_id")
repos_lang_committed.dropna(subset=['language'],inplace = True)
mydict = repos_lang_committed.groupby('committer_id').apply(lambda x: x['language'].tolist()).to_dict()
top_lang_committer = pd.Series(mydict).reset_index()
top_lang_committer.columns = ['user_id', 'languages']
top_lang_committer['top_lang_committed'] = top_lang_committer.languages.apply(toplanguages)
top_lang_committer.head()

,user_id,languages,top_lang_committed
0,21,"[go, go, go, go, go, go, go, go, go, go, go, g...","[go, ruby]"
1,25,"[ruby, ruby, ruby, ruby, ruby, ruby, css, ruby...","[ruby, coffeescript, css]"
2,38,"[ruby, ruby, ruby, ruby, ruby, coffeescript, r...","[ruby, coffeescript]"
3,108,"[ruby, html, ruby, ruby]","[ruby, html]"
4,173,"[ruby, ruby, css, ruby, ruby, ruby, ruby, coff...","[ruby, css, coffeescript, javascript, python, go]"


In [56]:
df_watched_reposx = pd.merge(df_watched_repos,df_repos[['repo_id','language']],how="left", on=['repo_id'])
repos_lang_watched = df_watched_reposx[df_watched_reposx['user_id'].isin(list(df_users.id))][['repo_id','user_id','language']]
repos_lang_watched = repos_lang_watched.sort_values("user_id")
repos_lang_watched.dropna(subset=['language'],inplace = True)
mydict = repos_lang_watched.groupby('user_id').apply(lambda x: x['language'].tolist()).to_dict()
top_lang_watcher = pd.Series(mydict).reset_index()
top_lang_watcher.columns = ['user_id', 'languages']
top_lang_watcher['top_lang_watched'] = top_lang_watcher.languages.apply(toplanguages)
top_lang_watcher.head()

,user_id,languages,top_lang_watched
0,21,"[ruby, ruby, javascript, go, javascript, c++, ...","[ruby, javascript, go, c, coffeescript, c++, p..."
1,25,"[objective-c, ruby, javascript, ruby, javascri...","[javascript, ruby, objective-c, python, coffee..."
2,38,"[ruby, ruby, javascript, ruby, ruby, ruby, rub...","[ruby, javascript, go, shell, c, coffeescript,..."
3,108,"[ruby, go, javascript, ruby, javascript, go, r...","[ruby, javascript, go, shell, objective-c, c, ..."
4,173,"[ruby, javascript, ruby, coffeescript, javascr...","[ruby, javascript, css, c, coffeescript, go, h..."


In [57]:
temp = pd.merge(top_lang_owner[['user_id','top_lang_owned']],
                              top_lang_committer[['user_id','top_lang_committed']],
                              how="left", on=['user_id'])
df_users_topLangs = pd.merge(temp,
                              top_lang_watcher[['user_id','top_lang_watched']],
                              how="left", on=['user_id'])

In [58]:
df_users_topLangs.head()

,user_id,top_lang_owned,top_lang_committed,top_lang_watched
0,21,"[ruby, javascript, go, coffeescript, c, c#, c+...","[go, ruby]","[ruby, javascript, go, c, coffeescript, c++, p..."
1,25,"[javascript, ruby, coffeescript, objective-c, ...","[ruby, coffeescript, css]","[javascript, ruby, objective-c, python, coffee..."
2,38,"[ruby, javascript, coffeescript, shell, go, ob...","[ruby, coffeescript]","[ruby, javascript, go, shell, c, coffeescript,..."
3,108,"[ruby, shell, go, javascript, css, coffeescrip...","[ruby, html]","[ruby, javascript, go, shell, objective-c, c, ..."
4,173,"[ruby, javascript, shell, coffeescript, c, css...","[ruby, css, coffeescript, javascript, python, go]","[ruby, javascript, css, c, coffeescript, go, h..."


In [59]:
temp = pd.merge(features_total, df_repos[['repo_id','language']], how="left", on=['repo_id'])
temp = pd.merge(temp, df_users_topLangs, how="left", on=['user_id'])

In [60]:
def func(x):
    if x.language in x.top_lang_owned:
        return 1
    return 0

In [61]:
temp['isin_top_lang_owned'] = temp[~temp[['language','top_lang_owned']].isna().any(1)].apply(func, axis=1)
temp.head()

,repo_id,user_id,is_ownered,num_commit_committed,num_commit_authored,num_commit_commented,num_issue_opened,num_issue_closed,num_issue_assigned,num_issue_commented,...,O_num_issue_commented,O_num_pr_opened,O_num_pr_assigned,O_num_pr_merged,O_num_pr_commented,language,top_lang_owned,top_lang_committed,top_lang_watched,isin_top_lang_owned
0,1,21,0,NaN,NaN,NaN,2.0,NaN,NaN,17.0,...,0.034623,0.009804,NaN,NaN,NaN,ruby,"[ruby, javascript, go, coffeescript, c, c#, c+...","[go, ruby]","[ruby, javascript, go, c, coffeescript, c++, p...",1.0
1,1,25,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,ruby,"[javascript, ruby, coffeescript, objective-c, ...","[ruby, coffeescript, css]","[javascript, ruby, objective-c, python, coffee...",1.0
2,1,38,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,ruby,"[ruby, javascript, coffeescript, shell, go, ob...","[ruby, coffeescript]","[ruby, javascript, go, shell, c, coffeescript,...",1.0
3,1,108,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,ruby,"[ruby, shell, go, javascript, css, coffeescrip...","[ruby, html]","[ruby, javascript, go, shell, objective-c, c, ...",1.0
4,1,173,0,NaN,NaN,NaN,2.0,NaN,NaN,136.0,...,0.276986,0.019608,NaN,0.098039,0.05,ruby,"[ruby, javascript, shell, coffeescript, c, css...","[ruby, css, coffeescript, javascript, python, go]","[ruby, javascript, css, c, coffeescript, go, h...",1.0


In [62]:
def func(x):
    if x.language in x.top_lang_committed:
        return 1
    return 0

In [63]:
temp['isin_top_lang_committed'] = temp[~temp[['language','top_lang_committed']].isna().any(1)].apply(func, axis=1)
temp.head()

,repo_id,user_id,is_ownered,num_commit_committed,num_commit_authored,num_commit_commented,num_issue_opened,num_issue_closed,num_issue_assigned,num_issue_commented,...,O_num_pr_opened,O_num_pr_assigned,O_num_pr_merged,O_num_pr_commented,language,top_lang_owned,top_lang_committed,top_lang_watched,isin_top_lang_owned,isin_top_lang_committed
0,1,21,0,NaN,NaN,NaN,2.0,NaN,NaN,17.0,...,0.009804,NaN,NaN,NaN,ruby,"[ruby, javascript, go, coffeescript, c, c#, c+...","[go, ruby]","[ruby, javascript, go, c, coffeescript, c++, p...",1.0,1.0
1,1,25,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,ruby,"[javascript, ruby, coffeescript, objective-c, ...","[ruby, coffeescript, css]","[javascript, ruby, objective-c, python, coffee...",1.0,1.0
2,1,38,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,ruby,"[ruby, javascript, coffeescript, shell, go, ob...","[ruby, coffeescript]","[ruby, javascript, go, shell, c, coffeescript,...",1.0,1.0
3,1,108,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,ruby,"[ruby, shell, go, javascript, css, coffeescrip...","[ruby, html]","[ruby, javascript, go, shell, objective-c, c, ...",1.0,1.0
4,1,173,0,NaN,NaN,NaN,2.0,NaN,NaN,136.0,...,0.019608,NaN,0.098039,0.05,ruby,"[ruby, javascript, shell, coffeescript, c, css...","[ruby, css, coffeescript, javascript, python, go]","[ruby, javascript, css, c, coffeescript, go, h...",1.0,1.0


In [64]:
def func(x):
    if x.language in x.top_lang_watched:
        return 1
    return 0

In [65]:
temp['isin_top_lang_watched'] = temp[~temp[['language','top_lang_watched']].isna().any(1)].apply(func, axis=1)
temp.head()

,repo_id,user_id,is_ownered,num_commit_committed,num_commit_authored,num_commit_commented,num_issue_opened,num_issue_closed,num_issue_assigned,num_issue_commented,...,O_num_pr_assigned,O_num_pr_merged,O_num_pr_commented,language,top_lang_owned,top_lang_committed,top_lang_watched,isin_top_lang_owned,isin_top_lang_committed,isin_top_lang_watched
0,1,21,0,NaN,NaN,NaN,2.0,NaN,NaN,17.0,...,NaN,NaN,NaN,ruby,"[ruby, javascript, go, coffeescript, c, c#, c+...","[go, ruby]","[ruby, javascript, go, c, coffeescript, c++, p...",1.0,1.0,1.0
1,1,25,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,ruby,"[javascript, ruby, coffeescript, objective-c, ...","[ruby, coffeescript, css]","[javascript, ruby, objective-c, python, coffee...",1.0,1.0,1.0
2,1,38,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,ruby,"[ruby, javascript, coffeescript, shell, go, ob...","[ruby, coffeescript]","[ruby, javascript, go, shell, c, coffeescript,...",1.0,1.0,1.0
3,1,108,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,ruby,"[ruby, shell, go, javascript, css, coffeescrip...","[ruby, html]","[ruby, javascript, go, shell, objective-c, c, ...",1.0,1.0,1.0
4,1,173,0,NaN,NaN,NaN,2.0,NaN,NaN,136.0,...,NaN,0.098039,0.05,ruby,"[ruby, javascript, shell, coffeescript, c, css...","[ruby, css, coffeescript, javascript, python, go]","[ruby, javascript, css, c, coffeescript, go, h...",1.0,1.0,1.0


In [66]:
x = temp[['repo_id','user_id','isin_top_lang_owned','isin_top_lang_committed','isin_top_lang_watched']]
x

,repo_id,user_id,isin_top_lang_owned,isin_top_lang_committed,isin_top_lang_watched
0,1,21,1.0,1.0,1.0
1,1,25,1.0,1.0,1.0
2,1,38,1.0,1.0,1.0
3,1,108,1.0,1.0,1.0
4,1,173,1.0,1.0,1.0
...,...,...,...,...,...
4127995,38458452,3346407,1.0,0.0,1.0
4127996,38458452,3451712,1.0,1.0,1.0
4127997,38458452,4323180,1.0,0.0,1.0
4127998,38458452,4921183,1.0,1.0,1.0


In [67]:
features_plus_lang = pd.merge(features_total, x, how="left", on=['user_id','repo_id'])
features_plus_lang.head()

,repo_id,user_id,is_ownered,num_commit_committed,num_commit_authored,num_commit_commented,num_issue_opened,num_issue_closed,num_issue_assigned,num_issue_commented,...,O_num_issue_assigned,O_num_issue_closedPR,O_num_issue_commented,O_num_pr_opened,O_num_pr_assigned,O_num_pr_merged,O_num_pr_commented,isin_top_lang_owned,isin_top_lang_committed,isin_top_lang_watched
0,1,21,0,NaN,NaN,NaN,2.0,NaN,NaN,17.0,...,NaN,0.053476,0.034623,0.009804,NaN,NaN,NaN,1.0,1.0,1.0
1,1,25,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0
2,1,38,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0
3,1,108,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0
4,1,173,0,NaN,NaN,NaN,2.0,NaN,NaN,136.0,...,NaN,0.106952,0.276986,0.019608,NaN,0.098039,0.05,1.0,1.0,1.0


In [68]:
features_plus_lang.drop(['is_ownered'], axis=1,inplace=True)
features_plus_lang.drop(['total_commit'], axis=1,inplace=True)
features_plus_lang.drop(['total_commit_comment'], axis=1,inplace=True)
features_plus_lang.drop(['total_issue'], axis=1,inplace=True)
features_plus_lang.drop(['total_issue_comment'], axis=1,inplace=True)
features_plus_lang.drop(['total_pr'], axis=1,inplace=True)
features_plus_lang.drop(['total_pr_comment'], axis=1,inplace=True)

In [69]:
features_plus_lang.head()

,repo_id,user_id,is_ownered,num_commit_committed,num_commit_authored,num_commit_commented,num_issue_opened,num_issue_closed,num_issue_assigned,num_issue_commented,...,O_num_issue_assigned,O_num_issue_closedPR,O_num_issue_commented,O_num_pr_opened,O_num_pr_assigned,O_num_pr_merged,O_num_pr_commented,isin_top_lang_owned,isin_top_lang_committed,isin_top_lang_watched
0,1,21,0,NaN,NaN,NaN,2.0,NaN,NaN,17.0,...,NaN,0.053476,0.034623,0.009804,NaN,NaN,NaN,1.0,1.0,1.0
1,1,25,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0
2,1,38,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0
3,1,108,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0
4,1,173,0,NaN,NaN,NaN,2.0,NaN,NaN,136.0,...,NaN,0.106952,0.276986,0.019608,NaN,0.098039,0.05,1.0,1.0,1.0


In [71]:
df = features_plus_lang.copy()
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4128000 entries, 0 to 4127999
Data columns (total 31 columns):
 #   Column                   Dtype  
---  ------                   -----  
 0   repo_id                  int64  
 1   user_id                  int64  
 2   is_ownered               int64  
 3   num_commit_committed     float64
 4   num_commit_authored      float64
 5   num_commit_commented     float64
 6   num_issue_opened         float64
 7   num_issue_closed         float64
 8   num_issue_assigned       float64
 9   num_issue_commented      float64
 10  num_issue_closedwithPR   float64
 11  num_pr_opened            float64
 12  num_pr_merged            float64
 13  num_pr_assigned          float64
 14  num_pr_commented         float64
 15  count                    float64
 16  O_num_commit_committed   float64
 17  O_num_commit_authored    float64
 18  O_num_commit_commented   float64
 19  O_num_issue_opened       float64
 20  O_num_issue_closed       float64
 21  O_num_is

In [72]:
df['code_contributions'] = (
    df.num_pr_opened.fillna(0) +
    df.num_issue_opened.fillna(0) +
    df.num_issue_closedwithPR.fillna(0) +
    df.num_pr_merged.fillna(0) +
    df.num_issue_assigned.fillna(0) +
    df.num_commit_committed.fillna(0) +
    df.num_pr_assigned.fillna(0) +
    df.num_issue_closed.fillna(0))

df.head()

,repo_id,user_id,is_ownered,num_commit_committed,num_commit_authored,num_commit_commented,num_issue_opened,num_issue_closed,num_issue_assigned,num_issue_commented,...,O_num_issue_closedPR,O_num_issue_commented,O_num_pr_opened,O_num_pr_assigned,O_num_pr_merged,O_num_pr_commented,isin_top_lang_owned,isin_top_lang_committed,isin_top_lang_watched,code_contributions
0,1,21,0,NaN,NaN,NaN,2.0,NaN,NaN,17.0,...,0.053476,0.034623,0.009804,NaN,NaN,NaN,1.0,1.0,1.0,4.0
1,1,25,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,0.0
2,1,38,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,0.0
3,1,108,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,0.0
4,1,173,0,NaN,NaN,NaN,2.0,NaN,NaN,136.0,...,0.106952,0.276986,0.019608,NaN,0.098039,0.05,1.0,1.0,1.0,7.0


In [73]:
df['comments'] = (
    df.num_issue_commented.fillna(0) + 
    df.num_commit_commented.fillna(0) +
    df.num_pr_commented.fillna(0) )

df.head()

,repo_id,user_id,is_ownered,num_commit_committed,num_commit_authored,num_commit_commented,num_issue_opened,num_issue_closed,num_issue_assigned,num_issue_commented,...,O_num_issue_commented,O_num_pr_opened,O_num_pr_assigned,O_num_pr_merged,O_num_pr_commented,isin_top_lang_owned,isin_top_lang_committed,isin_top_lang_watched,code_contributions,comments
0,1,21,0,NaN,NaN,NaN,2.0,NaN,NaN,17.0,...,0.034623,0.009804,NaN,NaN,NaN,1.0,1.0,1.0,4.0,17.0
1,1,25,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,0.0,0.0
2,1,38,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,0.0,0.0
3,1,108,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,0.0,0.0
4,1,173,0,NaN,NaN,NaN,2.0,NaN,NaN,136.0,...,0.276986,0.019608,NaN,0.098039,0.05,1.0,1.0,1.0,7.0,137.0


In [75]:
df['commit_2_comment'] = df.num_commit_committed.fillna(0) / df.num_commit_commented.fillna(0)
df['issue_2_comment'] = df.num_issue_opened.fillna(0) / df.num_issue_commented.fillna(0)
df['pr_2_comment'] = df.num_pr_opened.fillna(0) / df.num_pr_commented.fillna(0) 
df['code_2_comment'] = df.commit_2_comment.fillna(0) + df.issue_2_comment.fillna(0) + df.pr_2_comment.fillna(0)


df.loc[~np.isfinite(df['commit_2_comment']), 'commit_2_comment'] = np.nan
df.loc[~np.isfinite(df['issue_2_comment']), 'issue_2_comment'] = np.nan 
df.loc[~np.isfinite(df['pr_2_comment']), 'pr_2_comment'] = np.nan 
df.loc[~np.isfinite(df['code_2_comment']), 'code_2_comment'] = np.nan
df['comment_2_code'] =1 / df['code_2_comment']
df.loc[~np.isfinite(df['comment_2_code']), 'comment_2_code'] = np.nan
df.head()

,repo_id,user_id,is_ownered,num_commit_committed,num_commit_authored,num_commit_commented,num_issue_opened,num_issue_closed,num_issue_assigned,num_issue_commented,...,isin_top_lang_owned,isin_top_lang_committed,isin_top_lang_watched,code_contributions,comments,commit_2_comment,issue_2_comment,pr_2_comment,code_2_comment,comment_2_code
0,1,21,0,NaN,NaN,NaN,2.0,NaN,NaN,17.0,...,1.0,1.0,1.0,4.0,17.0,NaN,0.117647,NaN,NaN,NaN
1,1,25,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0,1.0,1.0,0.0,0.0,NaN,NaN,NaN,0.000000,NaN
2,1,38,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0,1.0,1.0,0.0,0.0,NaN,NaN,NaN,0.000000,NaN
3,1,108,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0,1.0,1.0,0.0,0.0,NaN,NaN,NaN,0.000000,NaN
4,1,173,0,NaN,NaN,NaN,2.0,NaN,NaN,136.0,...,1.0,1.0,1.0,7.0,137.0,NaN,0.014706,2.0,2.014706,0.49635


In [76]:
df['issue_related'] = (
    df.num_issue_commented.fillna(0) + 
    df.num_issue_opened.fillna(0) +
    df.num_issue_closed.fillna(0) +
    df.num_issue_assigned.fillna(0) +
    df.num_issue_closedwithPR.fillna(0))

df.head()

,repo_id,user_id,is_ownered,num_commit_committed,num_commit_authored,num_commit_commented,num_issue_opened,num_issue_closed,num_issue_assigned,num_issue_commented,...,isin_top_lang_committed,isin_top_lang_watched,code_contributions,comments,commit_2_comment,issue_2_comment,pr_2_comment,code_2_comment,comment_2_code,issue_related
0,1,21,0,NaN,NaN,NaN,2.0,NaN,NaN,17.0,...,1.0,1.0,4.0,17.0,NaN,0.117647,NaN,NaN,NaN,20.0
1,1,25,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0,1.0,0.0,0.0,NaN,NaN,NaN,0.000000,NaN,0.0
2,1,38,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0,1.0,0.0,0.0,NaN,NaN,NaN,0.000000,NaN,0.0
3,1,108,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0,1.0,0.0,0.0,NaN,NaN,NaN,0.000000,NaN,0.0
4,1,173,0,NaN,NaN,NaN,2.0,NaN,NaN,136.0,...,1.0,1.0,7.0,137.0,NaN,0.014706,2.0,2.014706,0.49635,140.0


In [77]:
df['pr_related'] = (
    df.num_pr_assigned.fillna(0) + 
    df.num_pr_commented.fillna(0) +
    df.num_pr_merged.fillna(0) + 
    df.num_pr_opened.fillna(0) )

df.head()

,repo_id,user_id,is_ownered,num_commit_committed,num_commit_authored,num_commit_commented,num_issue_opened,num_issue_closed,num_issue_assigned,num_issue_commented,...,isin_top_lang_watched,code_contributions,comments,commit_2_comment,issue_2_comment,pr_2_comment,code_2_comment,comment_2_code,issue_related,pr_related
0,1,21,0,NaN,NaN,NaN,2.0,NaN,NaN,17.0,...,1.0,4.0,17.0,NaN,0.117647,NaN,NaN,NaN,20.0,1.0
1,1,25,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0,0.0,0.0,NaN,NaN,NaN,0.000000,NaN,0.0,0.0
2,1,38,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0,0.0,0.0,NaN,NaN,NaN,0.000000,NaN,0.0,0.0
3,1,108,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0,0.0,0.0,NaN,NaN,NaN,0.000000,NaN,0.0,0.0
4,1,173,0,NaN,NaN,NaN,2.0,NaN,NaN,136.0,...,1.0,7.0,137.0,NaN,0.014706,2.0,2.014706,0.49635,140.0,4.0


In [78]:
df['commit_related'] = (
    df.num_commit_authored.fillna(0) + 
    df.num_commit_commented.fillna(0) +
    df.num_commit_committed.fillna(0) )

df.head()

,repo_id,user_id,is_ownered,num_commit_committed,num_commit_authored,num_commit_commented,num_issue_opened,num_issue_closed,num_issue_assigned,num_issue_commented,...,code_contributions,comments,commit_2_comment,issue_2_comment,pr_2_comment,code_2_comment,comment_2_code,issue_related,pr_related,commit_related
0,1,21,0,NaN,NaN,NaN,2.0,NaN,NaN,17.0,...,4.0,17.0,NaN,0.117647,NaN,NaN,NaN,20.0,1.0,0.0
1,1,25,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,NaN,NaN,NaN,0.000000,NaN,0.0,0.0,0.0
2,1,38,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,NaN,NaN,NaN,0.000000,NaN,0.0,0.0,0.0
3,1,108,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,NaN,NaN,NaN,0.000000,NaN,0.0,0.0,0.0
4,1,173,0,NaN,NaN,NaN,2.0,NaN,NaN,136.0,...,7.0,137.0,NaN,0.014706,2.0,2.014706,0.49635,140.0,4.0,0.0


In [86]:
df_binary = df.loc[:,'repo_id':'isin_top_lang_watched'].copy()
col_listb= list(df_binary)
col_listb.remove('repo_id')
col_listb.remove('user_id')
df_binary[col_listb]=np.where(df_binary[col_listb]>0,1,df_binary[col_listb])
df_binary.head()

,repo_id,user_id,num_commit_committed,num_commit_authored,num_commit_commented,num_issue_opened,num_issue_closed,num_issue_assigned,num_issue_commented,num_issue_closedwithPR,...,O_num_issue_assigned,O_num_issue_closedPR,O_num_issue_commented,O_num_pr_opened,O_num_pr_assigned,O_num_pr_merged,O_num_pr_commented,isin_top_lang_owned,isin_top_lang_committed,isin_top_lang_watched
0,1,21,NaN,NaN,NaN,1.0,NaN,NaN,1.0,1.0,...,NaN,1.0,1.0,1.0,NaN,NaN,NaN,1.0,1.0,1.0
1,1,25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0
2,1,38,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0
3,1,108,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0
4,1,173,NaN,NaN,NaN,1.0,NaN,NaN,1.0,1.0,...,NaN,1.0,1.0,1.0,NaN,1.0,1.0,1.0,1.0,1.0


In [87]:
df_binary.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4128000 entries, 0 to 4127999
Data columns (total 30 columns):
 #   Column                   Dtype  
---  ------                   -----  
 0   repo_id                  int64  
 1   user_id                  int64  
 2   num_commit_committed     float64
 3   num_commit_authored      float64
 4   num_commit_commented     float64
 5   num_issue_opened         float64
 6   num_issue_closed         float64
 7   num_issue_assigned       float64
 8   num_issue_commented      float64
 9   num_issue_closedwithPR   float64
 10  num_pr_opened            float64
 11  num_pr_merged            float64
 12  num_pr_assigned          float64
 13  num_pr_commented         float64
 14  count                    float64
 15  O_num_commit_committed   float64
 16  O_num_commit_authored    float64
 17  O_num_commit_commented   float64
 18  O_num_issue_opened       float64
 19  O_num_issue_closed       float64
 20  O_num_issue_assigned     float64
 21  O_num_is

In [89]:
collist = list(df_binary.loc[:,'num_commit_committed':'num_pr_commented'])
df['binary_count'] = df_binary[collist].sum(axis=1)
df.head()

,repo_id,user_id,num_commit_committed,num_commit_authored,num_commit_commented,num_issue_opened,num_issue_closed,num_issue_assigned,num_issue_commented,num_issue_closedwithPR,...,comments,commit_2_comment,issue_2_comment,pr_2_comment,code_2_comment,comment_2_code,issue_related,pr_related,commit_related,binary_count
0,1,21,NaN,NaN,NaN,2.0,NaN,NaN,17.0,1.0,...,17.0,NaN,0.117647,NaN,NaN,NaN,20.0,1.0,0.0,4.0
1,1,25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,NaN,NaN,NaN,0.000000,NaN,0.0,0.0,0.0,0.0
2,1,38,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,NaN,NaN,NaN,0.000000,NaN,0.0,0.0,0.0,0.0
3,1,108,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,NaN,NaN,NaN,0.000000,NaN,0.0,0.0,0.0,0.0
4,1,173,NaN,NaN,NaN,2.0,NaN,NaN,136.0,2.0,...,137.0,NaN,0.014706,2.0,2.014706,0.49635,140.0,4.0,0.0,6.0


In [90]:
df['binary_code_contributions'] = (
    df_binary.num_pr_opened.fillna(0) +
    df_binary.num_issue_opened.fillna(0) +
    df_binary.num_issue_closedwithPR.fillna(0) +
    df_binary.num_pr_merged.fillna(0) +
    df_binary.num_issue_assigned.fillna(0) +
    df_binary.num_commit_committed.fillna(0) +
    df_binary.num_pr_assigned.fillna(0) +
    df_binary.num_issue_closed.fillna(0))

df.head()

,repo_id,user_id,num_commit_committed,num_commit_authored,num_commit_commented,num_issue_opened,num_issue_closed,num_issue_assigned,num_issue_commented,num_issue_closedwithPR,...,commit_2_comment,issue_2_comment,pr_2_comment,code_2_comment,comment_2_code,issue_related,pr_related,commit_related,binary_count,binary_code_contributions
0,1,21,NaN,NaN,NaN,2.0,NaN,NaN,17.0,1.0,...,NaN,0.117647,NaN,NaN,NaN,20.0,1.0,0.0,4.0,3.0
1,1,25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.000000,NaN,0.0,0.0,0.0,0.0,0.0
2,1,38,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.000000,NaN,0.0,0.0,0.0,0.0,0.0
3,1,108,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.000000,NaN,0.0,0.0,0.0,0.0,0.0
4,1,173,NaN,NaN,NaN,2.0,NaN,NaN,136.0,2.0,...,NaN,0.014706,2.0,2.014706,0.49635,140.0,4.0,0.0,6.0,4.0


In [91]:
df['binary_comments'] = (
    df_binary.num_issue_commented.fillna(0) + 
    df_binary.num_commit_commented.fillna(0) +
    df_binary.num_pr_commented.fillna(0) )

df.head()

,repo_id,user_id,num_commit_committed,num_commit_authored,num_commit_commented,num_issue_opened,num_issue_closed,num_issue_assigned,num_issue_commented,num_issue_closedwithPR,...,issue_2_comment,pr_2_comment,code_2_comment,comment_2_code,issue_related,pr_related,commit_related,binary_count,binary_code_contributions,binary_comments
0,1,21,NaN,NaN,NaN,2.0,NaN,NaN,17.0,1.0,...,0.117647,NaN,NaN,NaN,20.0,1.0,0.0,4.0,3.0,1.0
1,1,25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.000000,NaN,0.0,0.0,0.0,0.0,0.0,0.0
2,1,38,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.000000,NaN,0.0,0.0,0.0,0.0,0.0,0.0
3,1,108,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.000000,NaN,0.0,0.0,0.0,0.0,0.0,0.0
4,1,173,NaN,NaN,NaN,2.0,NaN,NaN,136.0,2.0,...,0.014706,2.0,2.014706,0.49635,140.0,4.0,0.0,6.0,4.0,2.0


In [92]:
df['binary_issue_related'] = (
    df_binary.num_issue_commented.fillna(0) + 
    df_binary.num_issue_opened.fillna(0) +
    df_binary.num_issue_closed.fillna(0) +
    df_binary.num_issue_assigned.fillna(0) +
    df_binary.num_issue_closedwithPR.fillna(0))

df.head()

,repo_id,user_id,num_commit_committed,num_commit_authored,num_commit_commented,num_issue_opened,num_issue_closed,num_issue_assigned,num_issue_commented,num_issue_closedwithPR,...,pr_2_comment,code_2_comment,comment_2_code,issue_related,pr_related,commit_related,binary_count,binary_code_contributions,binary_comments,binary_issue_related
0,1,21,NaN,NaN,NaN,2.0,NaN,NaN,17.0,1.0,...,NaN,NaN,NaN,20.0,1.0,0.0,4.0,3.0,1.0,3.0
1,1,25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.000000,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1,38,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.000000,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1,108,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.000000,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1,173,NaN,NaN,NaN,2.0,NaN,NaN,136.0,2.0,...,2.0,2.014706,0.49635,140.0,4.0,0.0,6.0,4.0,2.0,3.0


In [93]:
df['binary_pr_related'] = (
    df_binary.num_pr_assigned.fillna(0) + 
    df_binary.num_pr_commented.fillna(0) +
    df_binary.num_pr_merged.fillna(0) + 
    df_binary.num_pr_opened.fillna(0) )

df.head()

,repo_id,user_id,num_commit_committed,num_commit_authored,num_commit_commented,num_issue_opened,num_issue_closed,num_issue_assigned,num_issue_commented,num_issue_closedwithPR,...,code_2_comment,comment_2_code,issue_related,pr_related,commit_related,binary_count,binary_code_contributions,binary_comments,binary_issue_related,binary_pr_related
0,1,21,NaN,NaN,NaN,2.0,NaN,NaN,17.0,1.0,...,NaN,NaN,20.0,1.0,0.0,4.0,3.0,1.0,3.0,1.0
1,1,25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.000000,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1,38,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.000000,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1,108,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.000000,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1,173,NaN,NaN,NaN,2.0,NaN,NaN,136.0,2.0,...,2.014706,0.49635,140.0,4.0,0.0,6.0,4.0,2.0,3.0,3.0


In [94]:
df['binary_commit_related'] = (
    df_binary.num_commit_authored.fillna(0) + 
    df_binary.num_commit_commented.fillna(0) +
    df_binary.num_commit_committed.fillna(0) )

df.head()

,repo_id,user_id,num_commit_committed,num_commit_authored,num_commit_commented,num_issue_opened,num_issue_closed,num_issue_assigned,num_issue_commented,num_issue_closedwithPR,...,comment_2_code,issue_related,pr_related,commit_related,binary_count,binary_code_contributions,binary_comments,binary_issue_related,binary_pr_related,binary_commit_related
0,1,21,NaN,NaN,NaN,2.0,NaN,NaN,17.0,1.0,...,NaN,20.0,1.0,0.0,4.0,3.0,1.0,3.0,1.0,0.0
1,1,25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1,38,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1,108,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1,173,NaN,NaN,NaN,2.0,NaN,NaN,136.0,2.0,...,0.49635,140.0,4.0,0.0,6.0,4.0,2.0,3.0,3.0,0.0


In [95]:
export_csv = df.to_csv (r'/Users/kadir/Desktop/All_Developer_Metrics.csv',index=False, header=True)